In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def dfdx(x,f):
    #declare contents of array
    y = f[0]
    z = f[1]
    
    dydx = np.zeros_like(f)
    dydx[0] = z
    dydx[1] = -y
    
    return dydx

In [ ]:
def cash_karp_core(xi, yi, nv, h, f):
    #I like Brant's notation here
    ni = 7
    nj = 6
    ci = np.zeros(ni)
    aij = np.zeros((ni, nj))
    bi = np.zeros(ni)
    bis = np.zeros(ni)
    
    #c ranges on 2
    ci[2] = 1.0/5.0
    ci[3] = 3.0/10.0
    ci[4] = 3.0/5.0
    ci[5] = 1.0
    ci[6] = 7.0/8.0
    #for all j=1
    aij[2,1] = 1.0/5.0
    aij[3,1] = 3.0/40.0
    aij[4,1] = 3.0/10.0
    aij[5,1] = -11.0/54.0
    aij[6,1] = 1631.0/55296.0
    #j = 2
    aij[3,2] = 9.0/40.0
    aij[4,2] = -9.0/10.0
    aij[5,2] = 5.0/2.0
    aij[6,2] = 175.0/512.0
    #j = 3
    aij[4,3] = 6.0/5.0
    aij[5,3] = -70.0/27.0
    aij[6,3] = 575.0/13824.0
    #j = 4
    aij[5,4] = 35.0/27.0
    aij[6,4] = 44275.0/110592.0
    #j = 5
    aij[6,5] = 253.0/4096.0
    
    #bi values
    bi[1] = 37.0/378.0
    bi[2] = 0.0
    bi[3] = 250.0/621.0
    bi[4] = 125.0/594.0
    bi[5] = 0.0
    bi[6] = 512.0/1771.0
    
    #bis values
    bis[1] = 2825.0/27648.0
    bis[2] = 0.0
    bis[3] = 18575.0/48384.0
    bis[4] = 13525.0/55296.0
    bis[5] = 277.0/14336.0
    bis[6] = 1.0/4.0
    
    #k[:] arrays
    ki = np.zeros((ni, nv))
    for i in range(1, ni):
        #this is the x_n+1 values
        xn = xi + (ci[i]*h)
        #this is the temporary y
        yn = yi.copy()
        for j in range(1, i):
            yn += aij[i,j]*ki[j,:]
        
        ki[i,:] = h*f(xn,yn)
       
    #im not quite sure what 'npo(s)' means but I will use Brant's notationa again
    ynpo = yi.copy()
    ynpos = yi.copy()
    
    for i in range(1, ni):
        ynpo += bi[i]*ki[i,:]
        ynpos += bis[i]*ki[i,:]
    
    error = np.fabs(ynpo-ynpos)
    
    return ynpo, error

In [ ]:
def cash_karp_adaptive_step(dfdx, xi, yi, nv, h, tol):
    #I will use Brant's safety scale
    
    safety = 0.9
    h_new = 2.0
    
    imax = 1000
    i = 0
    
    error = np.full(nv, 2*tol)
    
    #remember step for later
    h_step = h
    
    while(error.max()/tol > 1.0):
        yipo, error = cash_karp_core(xi, yi, nv, h_step, dfdx)
        
        if(error.max()/tol > 1.0):
            h_step *= safety*(error.max()/tol)**(-0.25)
        
        if(i >= imax):
            print("too many iterations")
            raise StopIteration("Ending after i = ", i)
        
        i += 1
    
    #now we try a bigger more efficient step
    h_new = np.fmin(h_step*(error.max()/tol)**(-0.9), h_step*h_new)
    
    return yipo, h_new, h_step

In [ ]:
def cash_karp(dfdx, a, b, ya, tol):
    xi = a
    yi = ya.copy()
    
    h = 1.0e-3*(b-a)
    
    imax = 1000
    i = 0
    nv = len(ya)
    
    #initial conditions
    x = np.full(1,a)
    y = np.full((1,nv), ya)
    
    flag = True
    
    while(flag):
        #calculate y_i+1
        yipo, h_new, h_step = cash_karp_adaptive_step(dfdx, xi, yi, nv, h, tol)
        
        h = h_new
        
        if(xi + h_step > b):
            h = b - xi
            
            #then recompute y_i+1
            yipo, h_new, h_step = cash_karp_adaptive_step(dfdx, xi, yi, nv, h, tol)
            
            flag = False
        
        #lets update our values
        xi += h_step
        yi = yipo.copy()
        
        x = np.append(x, xi)
        yipo = np.zeros((len(x),nv))
        yipo[0:len(x)-1,:] = y[:]
        yipo[-1,:] = yi[:]
        del y
        y = yipo
        
        #as always stop iterations
        if (i >= imax):
            print("max iterations reached")
            raise StopIteration("Iteration number =", i)
        
        i += 1
        
        if(xi == b):
            flag = False
            
    return x, y
        

In [ ]:
#lets perform the integration
a = 0.0
b = 2.0*np.pi
y0 = np.zeros(2)
y0[0] = 0.0
y0[1] = 1.0
nv = 2
tolerance = 1.0e-6

x, y = cash_karp(dfdx, a, b, y0, tolerance)

#I will copy Brant's plotting techniques as I had trouble with them
plt.plot(x,y[:,0], 'o', label = 'y(x)')
plt.plot(x,y[:,1], 'o', label = 'z(x)')
plt.plot(x, np.sin(x), label = 'sin(x)')
plt.plot(x, np.cos(x), label = 'cos(x)')
plt.xlim([0, 2*np.pi])
plt.ylim([-1.2, 1.2])
plt.legend(frameon = False)

In [ ]:
#now lets plot error
plt.plot(x, y[:,0]-np.sin(x), label = "y(x) error")
plt.plot(x, y[:,1]-np.cos(x), label = "z(x) error")